

```
# This is formatted as code
```

# Optical Character Recognition using Tesseract

### Step1. Install Pytesseract and tesseract-OCR in Google Colab.

In [7]:
!sudo apt install tesseract

Command not found


In [8]:
%pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Download the cascade file if not already present
!wget -O haarcascade_russian_plate_number.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_russian_plate_number.xml

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
import sys
print(sys.executable)


c:\Users\Akshat\AppData\Local\Programs\Python\Python313\python.exe


In [11]:
import cv2
import numpy as np

print(f"OpenCV version: {cv2.__version__}")
print(f"Numpy version: {np.__version__}")

OpenCV version: 4.11.0
Numpy version: 2.2.4


In [12]:
# import cv2
# import numpy as np

# def detect_and_crop_plate(image_path, cascade_path='haarcascade_russian_plate_number.xml'):
#     # Load the Haar cascade classifier for license plates
#     plate_cascade = cv2.CascadeClassifier(cascade_path)
#     if plate_cascade.empty():
#         print("Error loading cascade file. Check the path and file name.")
#         return

#     # Read the PNG image from the given path
#     image = cv2.imread(image_path)
#     if image is None:
#         print("Error reading image. Check the file path.")
#         return

#     # Convert the image to grayscale for detection
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # Detect number plates in the image
#     plates = plate_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, minSize=(30, 30))

#     if len(plates) == 0:
#         print("No license plates detected.")
#         return

#     # Process each detected plate (if multiple are found)
#     for i, (x, y, w, h) in enumerate(plates):
#         # Crop the detected license plate
#         cropped_plate = image[y:y+h, x:x+w]

#         # Display the cropped license plate using cv2_imshow
#         cv2.imshow(f'Cropped Plate {i+1}', cropped_plate)

#         # Save the cropped license plate
#         cv2.imwrite('cropped_plate.png', cropped_plate)
#         print(f'Cropped license plate saved as croppedplate.png')

# if __name__ == "__main__":
#     # Download the cascade file if not already present
#     # !wget -O haarcascade_russian_plate_number.xml https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_russian_plate_number.xml

#     # Replace with the correct path to your uploaded PNG file
#     image_path = 'D:/Academics/Semester VI/ES 333 - Microprocessors and Embedded Systems/PROJECT/license_plate_image.png'
#     detect_and_crop_plate(image_path)

### Step2. import libraries

In [13]:
import pytesseract
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image

In [14]:
import cv2
import numpy as np

#sharpen image
image = cv2.imread("D:/Academics/Semester VI/ES 333 - Microprocessors and Embedded Systems/PROJECT/license_plate_image.png")
sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
sharpened = cv2.filter2D(image, -1, sharpen_kernel)
cv2.imwrite("sharpened.png", sharpened)

#remove noise
gray = cv2.cvtColor(sharpened, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)
cv2.imwrite("thresh.png", thresh)

True

## Step4. Text Extraction

In [2]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [16]:
from PIL import Image
import pytesseract

# (optional) if you haven't set the path already
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

extractedInformation = pytesseract.image_to_string(Image.open('sharpened.png')).strip()

# Defensive check before accessing index 2
if len(extractedInformation) > 2 and extractedInformation[2] == 'O':
    extractedInformation = extractedInformation[:2] + '0' + extractedInformation[3:]

# Defensive check before accessing index 0
if len(extractedInformation) > 0 and not extractedInformation[0].isalpha():
    extractedInformation = extractedInformation[1:]

print(f"Extracted text: {extractedInformation}")

Extracted text: Python


### Step5. Printing the extracted information

In [17]:
print(extractedInformation)

Python


In [18]:
import cv2
import pytesseract
import numpy as np

# Load and grayscale
img = cv2.imread("license_plate_image.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Enhance contrast with CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
gray = clahe.apply(gray)

# Resize to improve OCR accuracy
resized = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

# Morph close to connect thin/broken lines
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
closed = cv2.morphologyEx(resized, cv2.MORPH_CLOSE, kernel)

# Optional: slight blur to suppress tiny noise
blurred = cv2.GaussianBlur(closed, (3, 3), 0)

# Optional: invert if text is white
if np.mean(blurred) < 127:
    blurred = cv2.bitwise_not(blurred)

# Save for debugging
cv2.imwrite("ocr_ready.png", blurred)

# Tesseract config
config = (
    r'--psm 7 --oem 3 '
    r'-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '
    r'-c load_system_dawg=0 -c load_freq_dawg=0'
)

# OCR
text = pytesseract.image_to_string(blurred, config=config).strip().replace(" ", "")
print("OCR Result:", text)

OCR Result: OO


In [19]:
import cv2
import pytesseract
import numpy as np

# Load the image
img = cv2.imread("license_plate_image.png")

# Rotate the image by 180 degrees
img = cv2.rotate(img, cv2.ROTATE_180)

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Enhance contrast with CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
gray = clahe.apply(gray)

# Resize to improve OCR accuracy
resized = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

# Morph close to connect thin/broken lines
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
closed = cv2.morphologyEx(resized, cv2.MORPH_CLOSE, kernel)

# Optional: slight blur to suppress tiny noise
blurred = cv2.GaussianBlur(closed, (3, 3), 0)

# Optional: invert if text is white
if np.mean(blurred) < 127:
    blurred = cv2.bitwise_not(blurred)

# Save for debugging
cv2.imwrite("ocr_ready.png", blurred)

# Tesseract config
config = (
    r'--psm 7 --oem 3 '
    r'-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '
    r'-c load_system_dawg=0 -c load_freq_dawg=0'
)

# OCR
text = pytesseract.image_to_string(blurred, config=config).strip().replace(" ", "")
print("OCR Result:", text)

OCR Result: 


In [20]:
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime

# Initialize Firebase Admin SDK
cred = credentials.Certificate("D:\Academics\Semester VI\ES 333 - Microprocessors and Embedded Systems\PROJECT\snippet.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://anpr-bg-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

def get_next_index(ref):
    """Get the next sequential integer key as a string."""
    existing_data = ref.get()
    if not existing_data:
        return "1"
    
    existing_keys = [int(key) for key in existing_data.keys() if key.isdigit()]
    if not existing_keys:
        return "1"
    
    return str(max(existing_keys) + 1)

def add_vehicle_entry(vehicle_number):
    """Adds an outsider vehicle entry with a timestamp and plate number under a sequential key."""
    ref = db.reference("outsiders")
    now = datetime.now().strftime("%H:%M:%S")  # Current time

    next_index = get_next_index(ref)
    
    ref.child(next_index).set({
        "entry": now,
        "plate": vehicle_number
    })

    print(f"Entry logged for {vehicle_number} at {now} under ID {next_index}")

# Get user input and log entry
vehicle_number = "MH12AG7654"
add_vehicle_entry(vehicle_number)

<>:6: SyntaxWarning: invalid escape sequence '\A'
<>:6: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Akshat\AppData\Local\Temp\ipykernel_22252\28749696.py:6: SyntaxWarning: invalid escape sequence '\A'
  cred = credentials.Certificate("D:\Academics\Semester VI\ES 333 - Microprocessors and Embedded Systems\PROJECT\snippet.json")


Entry logged for MH12AG7654 at 04:09:53 under ID 2


In [32]:
import cv2
import pytesseract
import numpy as np

# Load the image
img = cv2.imread("license_plate_image.png")

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply thresholding to preprocess the image
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# Perform text extraction
config = '--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
text = pytesseract.image_to_string(thresh, config=config)

# Clean up the text
clean_text = ''.join(e for e in text if e.isalnum())

print(f"Detected license plate: {clean_text}")

Detected license plate: 


In [30]:
import cv2
import pytesseract
import numpy as np

# Load the image
img = cv2.imread("license_plate_image.jpg")

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply multiple preprocessing techniques and try OCR on each
results = []

# 1. Basic thresholding
_, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
results.append(thresh1)

# 2. Adaptive thresholding
thresh2 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
results.append(thresh2)

# 3. Contrast enhancement
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced = clahe.apply(gray)
_, thresh3 = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
results.append(thresh3)

# 4. Denoising + thresholding
denoised = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)
_, thresh4 = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
results.append(thresh4)

# OCR configuration
config = '--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

# Try OCR on all preprocessed images
all_texts = []
for i, processed_img in enumerate(results):
    # Add border to help Tesseract
    processed_img = cv2.copyMakeBorder(processed_img, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=255)
    
    # Save intermediate results for debugging
    cv2.imwrite(f"processed_{i+1}.jpg", processed_img)
    
    # Perform OCR
    text = pytesseract.image_to_string(processed_img, config=config).strip()
    clean_text = ''.join(e for e in text if e.isalnum())
    
    if clean_text:
        all_texts.append(clean_text)
        print(f"Method {i+1} detected: {clean_text}")

# Choose the most common result or the first non-empty one
if all_texts:
    from collections import Counter
    most_common = Counter(all_texts).most_common(1)[0][0]
    print(f"\nFinal detected license plate: {most_common}")
else:
    print("No license plate detected with any method")

Method 4 detected: MHA5XY2468

Final detected license plate: MHA5XY2468
